<a href="https://colab.research.google.com/github/hemillyraiky/telecomx_alura_one/blob/main/TelecomX_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracão

In [ ]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url_dados = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/main/TelecomX_Data.json'
resposta = requests.get(url_dados)
dados_json = resposta.json()

In [ ]:
tabela_clientes = pd.json_normalize(dados_json)

In [ ]:
tabela_clientes.head()

#🔧 Transformação

In [ ]:
tabela_clientes.iloc[0]

In [ ]:
tabela_clientes.info()

In [ ]:
tabela_clientes.isnull().sum()
tabela_clientes.isnull().mean().sort_values(ascending=False)

In [ ]:
tabela_clientes.duplicated().sum()
tabela_clientes[tabela_clientes.duplicated()]

In [ ]:
tabela_clientes['account.Charges.Total'] = pd.to_numeric(
    tabela_clientes['account.Charges.Total'], errors='coerce')

In [ ]:
tabela_clientes['account.PaymentMethod'] = tabela_clientes['account.PaymentMethod'].str.strip().str.title()

In [ ]:
(tabela_clientes == '').sum()
tabela_clientes['Churn'] = tabela_clientes['Churn'].fillna('No')

In [ ]:
tabela_clientes['Fatura_Diaria'] = tabela_clientes['account.Charges.Monthly'] / 30

In [ ]:
tabela_clientes['customer.gender'] = tabela_clientes['customer.gender'].replace({'Female': 'Feminino','Male': 'Masculino'})

In [ ]:
colunas_binarias = [
    'Churn', 'customer.Partner', 'customer.Dependents', 'phone.PhoneService','phone.MultipleLines', 'internet.OnlineSecurity', 'internet.OnlineBackup','internet.DeviceProtection', 'internet.TechSupport',
    'internet.StreamingTV','internet.StreamingMovies', 'account.PaperlessBilling']

In [ ]:
valores_binarios = {
    'Yes': 1,
    'No': 0,
    '': 0,
    'No internet service': 0,
    'No phone service': 0
}

In [ ]:
for coluna in colunas_binarias:
    tabela_clientes[coluna] = tabela_clientes[coluna].apply(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
for coluna in colunas_binarias:
    tabela_clientes[coluna] = (tabela_clientes[coluna].apply(lambda x: x.strip() if isinstance(x, str) else x).replace(valores_binarios).astype('Int64'))

In [ ]:

tabela_clientes.rename(columns={
    'customer.gender': 'Gênero',
    'customer.SeniorCitizen': 'Idoso',
    'customer.Partner': 'Possui Parceiro',
    'customer.Dependents': 'Possui Dependentes',
    'customer.tenure': 'Meses de Contrato',
    'phone.PhoneService': 'Telefone Ativo',
    'phone.MultipleLines': 'Mais de uma Linha',
    'internet.InternetService': 'Tipo de Internet',
    'internet.OnlineSecurity': 'Segurança Online',
    'internet.OnlineBackup': 'Backup Online',
    'internet.DeviceProtection': 'Proteção de Dispositivo',
    'internet.TechSupport': 'Suporte Técnico',
    'internet.StreamingTV': 'Streaming de TV',
    'internet.StreamingMovies': 'Streaming de Filmes',
    'account.Contract': 'Tipo de Contrato',
    'account.PaperlessBilling': 'Fatura Digital',
    'account.PaymentMethod': 'Forma de Pagamento',
    'account.Charges.Monthly': 'Valor Mensal',
    'account.Charges.Total': 'Valor Total',
    'Fatura_Diaria': 'Valor Diário',
    'Churn': 'Churn'
}, inplace=True)



In [ ]:
tabela_clientes.describe()

#📊 Carga e análise

In [ ]:
cor_1 = '#2a9d8f'  # azul petróleo
cor_2 = '#e76f51'  # laranja queimado


In [ ]:
contagem_churn = tabela_clientes['Churn'].value_counts().sort_index()
plt.figure(figsize=(6, 4))
contagem_churn.plot(kind='bar', color=[cor_1, cor_2])
plt.title('Distribuição de Evasão (Churn)')
plt.xticks(ticks=[0, 1], labels=['Permaneceu', 'Saiu'], rotation=0)
plt.ylabel('Número de Clientes')
plt.xlabel('Churn')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()

In [ ]:
etiquetas = ['Permaneceu', 'Saiu']
valores = tabela_clientes['Churn'].value_counts().sort_index()
plt.figure(figsize=(5, 5))
plt.pie(valores, labels=etiquetas, autopct='%1.1f%%', colors=[cor_1, cor_2], startangle=90)
plt.title('Proporção de Clientes com Churn')
plt.axis('equal')

plt.show()

In [ ]:
def analisar_churn_categoria(nome_coluna):
    plt.figure(figsize=(8, 4))
    sns.countplot(data=tabela_clientes, x=nome_coluna, hue='Churn', palette='Set2')
    plt.title(f'Churn por {nome_coluna}')
    plt.xticks(rotation=0, ha='center')
    plt.ylabel('Número de Clientes')
    plt.xlabel(nome_coluna)
    plt.legend(title='Churn', labels=['Permaneceu (0)', 'Saiu (1)'])
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
tabela_clientes.columns


In [ ]:
analisar_churn_categoria('Forma de Pagamento')
analisar_churn_categoria('Tipo de Contrato')
analisar_churn_categoria('Tipo de Internet')
analisar_churn_categoria('Idoso')
analisar_churn_categoria('Possui Parceiro')



In [ ]:
def boxplot_churn_coluna(nome_coluna):
    plt.figure(figsize=(6, 4))
    sns.boxplot(data=tabela_clientes, x='Churn', y=nome_coluna, palette=[cor_1, cor_2])
    plt.title(f'{nome_coluna} x Churn')
    plt.xlabel('Churn (0 = Ficou, 1 = Saiu)')
    plt.ylabel(nome_coluna)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


In [ ]:
boxplot_churn_coluna('Meses de Contrato')
boxplot_churn_coluna('Valor Mensal')
boxplot_churn_coluna('Valor Total')
boxplot_churn_coluna('Valor Diário')


In [ ]:
correlacoes_com_churn = tabela_clientes.corr(numeric_only=True)['Churn'].sort_values(ascending=False)
print(correlacoes_com_churn)

# Ordenar colunas pela correlação com churn
colunas_ordenadas = correlacoes_com_churn.index.tolist()

# Criar matriz de correlação reordenada
matriz_correlacao = tabela_clientes[colunas_ordenadas].corr(numeric_only=True)

# Plotar heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(
    matriz_correlacao,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8}
)
plt.title('Matriz de Correlação entre Variáveis Numéricas')
plt.tight_layout()
plt.show()


#📄Relatorio Final


# Relatório Final – Análise de Evasão de Clientes da Telecom X

## 1. Introdução

Este relatório apresenta o resultado da análise de dados realizada no contexto do projeto **Telecom X**, com o objetivo de entender os fatores que influenciam a evasão de clientes (churn) em uma empresa de telecomunicações. O foco principal foi aplicar o processo de ETL (Extração, Transformação e Carga) e realizar uma análise exploratória de dados (EDA), a fim de gerar insights que ajudem na construção de modelos preditivos por parte da equipe de ciência de dados.

---

## 2. Problema

A **Telecom X** vem enfrentando uma taxa elevada de evasão de clientes, o que impacta diretamente sua receita e estabilidade no mercado. No entanto, os motivos exatos para essa evasão ainda são incertos. A proposta deste trabalho é preparar os dados de forma adequada e explorar possíveis padrões que possam justificar esse comportamento.

---

## 3. Processo de ETL

### 📌 Extração
Os dados foram extraídos via API diretamente do repositório público no GitHub:
```
https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/main/TelecomX_Data.json
```

### 🔧 Transformação
As etapas de transformação envolveram:

- Conversão de colunas numéricas com erros de formatação.
- Padronização de campos textuais (ex.: gênero, forma de pagamento).
- Tratamento de valores nulos e vazios.
- Criação de novas variáveis, como **Valor Diário da Fatura**.
- Conversão de colunas binárias ('Yes'/'No') em valores 1 e 0.
- Tradução e renomeação das colunas para facilitar a leitura.

### 📊 Carga e Análise
Com os dados tratados, foi realizada a análise exploratória com foco em:

#### Distribuição de Churn
- **27%** dos clientes deixaram a empresa.
- Gráficos de barra e pizza representaram essa distribuição.

#### Análise por Categoria
Foram analisados os impactos do churn em variáveis como:

- **Gênero** (sem grandes diferenças)
- **Tipo de Contrato**: contratos mensais estão mais associados ao churn.
- **Forma de Pagamento**: pagamentos eletrônicos e automáticos têm maior evasão.
- **Internet e Streaming**: clientes com serviços extras têm mais churn.

#### Análise Numérica (Boxplots)
- **Clientes com menos meses de contrato** tendem a sair mais.
- Faturas mensais e totais mais altas estão relacionadas com evasão.

#### Matriz de Correlação
- As variáveis mais correlacionadas com churn foram:
  - `Tipo de Contrato`
  - `Fatura Digital`
  - `Valor Mensal`
  - `Meses de Contrato` (inversamente)

---

## 4. Conclusão

A análise demonstrou que os principais fatores associados ao churn são:

- Baixo tempo de contrato
- Alta fatura mensal
- Pagamentos eletrônicos ou automatizados
- Contratos mensais e sem fidelização

Esses padrões podem ser explorados em futuros modelos preditivos para identificar clientes com maior risco de evasão.

---

📌 *Este relatório resume a primeira fase do projeto Telecom X, focando na estruturação e análise inicial dos dados. As próximas etapas poderão se beneficiar deste trabalho para tomada de decisão e desenvolvimento de soluções baseadas em dados.*
